In [1]:
import pandas as pd

In [2]:
trade_data = pd.read_csv('./HS92_ccp_2014.csv', dtype={'commoditycode': str} )
africa_iso_code = pd.read_csv('./africa_iso_codes.csv')
hs4_2digit = pd.read_csv('./hs92_atlas.csv', dtype={'code': str})
hs4_2digit = hs4_2digit[hs4_2digit['level'] == '2digit'][['name','code']]
china_iso_code = 'CHN'

In [3]:
len(trade_data)

6002452

In [4]:
africa_trade_data = trade_data.loc[
    (trade_data['importer'] == china_iso_code) & \
    (trade_data['exporter'].isin(africa_iso_code['3digit']))
]

In [5]:
len(africa_trade_data)

35528

In [6]:
## Cleaning because some dumbfuck decided that codes should be numbers and not strings,
## And some codes that had 0XXX became XXX.
## Good fucking Game

def mutate_code_to_4_digit(value):
    if len(value) > 4:
        return value
    return (4 - len(value)) *'0' + value
        
africa_trade_data = africa_trade_data.assign(commoditycode = lambda x: x['commoditycode'].map(mutate_code_to_4_digit))
africa_trade_data = africa_trade_data.assign(commodity_group = lambda x: x['commoditycode'].map(lambda y: y[0:2]))


In [7]:
data = africa_trade_data.groupby(['commodity_group', 'exporter']).sum()

In [8]:
data.tail(3)

year  export_value  import_value
commodity_group exporter                                  
99              ZAF       2014   25400000000     192000000
                ZMB       2014         73838        612368
                ZWE       2014             0        331579

In [9]:
data = data.reset_index()

In [10]:
data = data.merge(africa_iso_code, left_on='exporter', right_on='3digit', how='left')

In [11]:
data = data.merge(hs4_2digit, left_on='commodity_group', right_on='code')
data.columns

Index(['commodity_group', 'exporter', 'year', 'export_value', 'import_value',
       'name_x', '2digit', '3digit', 'name_y', 'code'],
      dtype='object')

In [12]:
data.rename(columns={'name_x': 'country_name', 'name_y': 'product_name'}, inplace=True)

In [13]:
data = data[['commodity_group','product_name', 'export_value', 'import_value','country_name', '2digit', '3digit']]

In [14]:
data.to_csv('./visualization_data.csv')